In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [6]:
qdf= pd.read_csv('../Raw_MoleculeNet/qm8/raw/qm8.csv')
smiles = qdf['smiles']

failed = []
failed_optim = []
count = 0
path = 'data/from-smiles/gdb8-3d/'
sdf_file = 'gdb8.sdf'
with Chem.SDWriter(path + sdf_file) as writer:
    for smile in smiles:
        try:
            mol = Chem.MolFromSmiles(smile)
            mol = Chem.AddHs(mol)
            mol.SetProp('_Name', smile)
            if (AllChem.EmbedMolecule(mol, randomSeed = 0xf00d, maxAttempts = 100) == -1):
                print(f'{smile} failed to embed molecule')
            # optim = AllChem.MMFFOptimizeMolecule(mol)
            # if optim == -1:
            #     failed_optim.append(smile)
            #     raise ValueError('Failed to optimize conformer')
            # elif optim == 1:
            #     print(f'{smile} requires more iterations to converge')
            # AllChem.Compute2DCoords(mol)
            writer.write(mol)
        except Exception as e:
            failed.append(smile)
            print(e)
            continue

qdf = qdf[~qdf['smiles'].isin(failed)]
csv_file = 'gdb8.sdf.csv'
qdf.drop(columns=['smiles'], inplace=False).to_csv(path + csv_file, index=False)

[H]C1([H])C([H])([H])C2([H])C([H])([H])C([H])([H])C12[H] failed to embed molecule
[H]C1([H])OC2([H])C([H])([H])C([H])([H])C12[H] failed to embed molecule
[H]C1([H])OC2([H])C([H])([H])OC12[H] failed to embed molecule
[H]C1([H])C2([H])C3([H])C([H])([H])C2([H])C13[H] failed to embed molecule
[H]C1([H])N2C3([H])C([H])([H])C2([H])C13[H] failed to embed molecule
[H]C1([H])C2([H])C3([H])OC2([H])C13[H] failed to embed molecule
[H]N1C2([H])C3([H])N([H])C2([H])C13[H] failed to embed molecule
[H]N1C2([H])C3([H])OC2([H])C13[H] failed to embed molecule
[H]C12OC3([H])C1([H])OC23[H] failed to embed molecule
[H]C1([H])N2C3([H])C([H])([H])C1([H])C23[H] failed to embed molecule
[H]C1([H])C2([H])C([H])([H])C3([H])C1([H])C23[H] failed to embed molecule
[H]C1([H])C2([H])N3C1([H])C3([H])C2([H])[H] failed to embed molecule
[H]C1([H])C2([H])OC3([H])C1([H])C23[H] failed to embed molecule
[H]OC1([H])C2([H])C([H])([H])C1(C([H])([H])[H])C2([H])[H] failed to embed molecule
[H]C([H])([H])C1([H])C2([H])OC1(C([H])([H

In [5]:
sdf = Chem.SDMolSupplier('data/from-smiles/gdb8-3d/gdb8.sdf')

In [22]:
try:
    mol = Chem.MolFromSmiles('[H]C([H])([H])C12C([H])([H])N(C1([H])[H])C2([H])[H]')
    mol = Chem.AddHs(mol)
    print(AllChem.EmbedMolecule(mol))
    print(AllChem.EmbedMolecule(mol))
except:
    print('Failed to generate conformer for unrecognized molecules')

-1
-1
